# "Optimization for Statistical Learning" Expirement Notebook
## Section 0 Introduction

### Section 0.1 Data Importing and Preprocessing
In this section, we include packages we will use in this notebook.

In [ ]:
# Install packages #
# !pip install jupyter
# !pip install numpy
# !pip install matplotlib
# !pip install math
# !pip install torch
# !pip install xlrd
# !pip install pandas
# !pip install torchvision

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import math
import copy
import random
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch import Tensor
from torch.optim.optimizer import (Optimizer, required, _use_grad_for_differentiable, _default_to_fused_or_foreach,
                        _differentiable_doc, _foreach_doc, _maximize_doc)
from typing import List, Optional
from torch.autograd import Variable
import timeit

**Load npy files to restore results in the past**

In [ ]:
#past_results = np.load('./result.npy', encoding = "latin1")
#print(past_results)

## Section 0.2 Global Classes, Functions and Algorithms

**Spliting the DataLoader**

In [ ]:
def split_dataloader(dataloader, num_piece):
    dataset = list(dataloader)
    subset_size = len(dataset) // num_piece
    remainder = len(dataset) % num_piece
    split_subsets = []
    start_idx = 0
    for i in range(num_piece):
        if i < remainder:
            end_idx = start_idx + subset_size + 1
        else:
            end_idx = start_idx + subset_size
        subset = dataset[start_idx:end_idx]
        split_subsets.append(subset)
        start_idx = end_idx
    split_dataloaders = []
    for subset in split_subsets:
        split_dataloader = torch.utils.data.DataLoader(subset, batch_size=dataloader.batch_size, shuffle=True)
        split_dataloaders.append(split_dataloader)
    return split_dataloaders

**Error Rate Analaysis**

In [ ]:
# Train and test error rate analysis function
def calculate_error_rate(X, y, predict):
    num_samples = X.shape[0]
    error_count = torch.count_nonzero(torch.round(predict) - y)
    error_rate = error_count / num_samples
    return error_rate

**Custom Optimizer Class of Vanilia Gradient Descent**

The name custom_optimizer_SGD is just for consistency with torch.optim.SGD

In [ ]:
class custom_optimizer_SGD(Optimizer):
    def __init__(self, params, lr=required, weight_decay=0 ):
        if lr is not required and lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if weight_decay < 0.0:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")
        defaults = dict(lr=lr, weight_decay=weight_decay)
        super().__init__(params, defaults)
                
    def step(self):
        for group in self.param_groups:
            for param in group['params']:
                if param.grad is None:
                    continue
                grad = param.grad.data
                weight_decay = group['weight_decay']
                lr = group['lr']
                param.data.add_(-lr, grad)
                if weight_decay != 0:
                    param.data.add_(-lr * weight_decay, param.data)

**Client Device Class and Federated Learning Algorithms**

In [ ]:
# Define a custom class for each client so they can update separately
class ClientDevice:
    def __init__(self, model, criterion, optimizer, X_train, y_train):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.X_train = X_train
        self.y_train = y_train
        self.num_samples = self.X_train.size()[0]
        self.num_features = self.X_train.size()[1]

    def load_global_weights(self, global_weights):
        self.model.load_state_dict(global_weights)

    def get_local_weights(self):
        return self.model.state_dict()

    def update_weights_GDVanilia(self, num_epochs):
        self.model.train()
        for epoch in range(num_epochs):
            # Update weight
            outputs = self.model(self.X_train.float())
            loss = self.criterion(outputs, self.y_train.float())
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        return self.model.state_dict()
    
    def update_weights_GDStochastic(self, num_epochs, batch_size):
        self.model.train()
        num_batches = self.num_samples // batch_size
        for epoch in range(num_epochs):
            # Shuffle the data for each epoch
            permutation = torch.randperm(self.num_samples)
            X_shuffled = self.X_train[permutation]
            y_shuffled = self.y_train[permutation]
            for batch in range(num_batches):
                # Select the current batch
                start = batch * batch_size
                end = (batch + 1) * batch_size
                X_batch = X_shuffled[start:end]
                y_batch = y_shuffled[start:end]
                # Update weight
                outputs = self.model(X_batch.float())
                loss = self.criterion(outputs, y_batch.float())
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
        return self.model.state_dict()
    
    def update_weights(self, num_epochs, input_shape, iterate_func):
        self.model.train()
        loss_history, error_history = iterate_func(self.model, self.train_loader, num_epochs, self.optimizer, self.criterion, input_shape, show_history=False, training=True)
        return self.model.state_dict(), loss_history, error_history

# Establish client devices
def establish_client_devices(num_clients, dataset_loader, model_list, optimizer_list, criterion_list):
    # Establish client devices
    client_device = [None] * num_clients
    client_model = [None] * num_clients
    client_optimizer = [None] * num_clients
    client_criterion = [None] * num_clients
    client_weights = [None] * num_clients
    for client in range(num_clients):
        client_model[client] = model_list[client]
        client_optimizer[client] = optimizer_list[client]
        client_criterion[client] = criterion_list[client]
        client_weights[client] = client_model[client].state_dict()
        client_device[client] = ClientDevice(client_model[client], client_optimizer[client], client_criterion[client], dataset_loader)
    return client_device
    

# Define server wise functions
def send_client_weights(server_weights, local_weights):
    server_weights.append(local_weights)

# Total weight processing functions
def Federated_Averaging(client_weights_total):
    total_clients = len(client_weights_total)
    aggregate_weights = {}

    # Initialize aggregate_weights with the first client's weights
    for layer_name, layer_weights in client_weights_total[0].items():
        aggregate_weights[layer_name] = layer_weights / total_clients

    # Aggregate weights from the remaining clients
    for client_weights in client_weights_total[1:]:
        for layer_name, layer_weights in client_weights.items():
            aggregate_weights[layer_name] += layer_weights / total_clients

    return aggregate_weights

## Section 1. Linear Training Model
In this section, we will focus on the dataset requires no complicated data processing, and mainly linear models.

### Section 1.0. Data Loading and Preprocessing
Here we load the data for the expriements.

**BMI Dataset**

In [ ]:
### BMI Dataset

# Loading training data
dataset = pd.read_csv("bmi_train.csv")
dataset.replace({'Gender': {'Female': 0, 'Male': 1}}, inplace=True) #Gender -> boolean
dataset = dataset.to_numpy()

# Splitting off 80% of data for training, 20% for validation
train_split = int(0.8 * len(dataset))
X_train = dataset[:train_split, [0,1,2]]
y_train = dataset[:train_split, 3]
X_test = dataset[train_split:, [0,1,2]]
y_test = dataset[train_split:, 3]
print(X_train)
print(y_train)

# Loading prediction data
prediction_dataset = pd.read_csv("bmi_validation.csv")
prediction_dataset.replace({'Gender': {'Female': 0, 'Male': 1}}, inplace=True) #Gender -> boolean
X_prediction = prediction_dataset.to_numpy()

# Normalize data set
X_train_normalized = (X_train - X_train.min(0)) / (X_train.max(0) - X_train.min(0))
X_test_normalized = (X_test - X_test.min(0)) / (X_test.max(0) - X_test.min(0))
X_prediction_normalized = (X_prediction - X_prediction.min(0)) / (X_prediction.max(0) - X_prediction.min(0))

# Turn data to tensor
X_train_tensor = torch.from_numpy(X_train_normalized)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test_normalized)
y_test_tensor = torch.from_numpy(y_test)
X_prediction_tensor = torch.from_numpy(X_prediction_normalized)
print(X_train_tensor.size())
print(y_train_tensor.size())
print(X_test_tensor.size())
print(y_test_tensor.size())

# Learning Rate and Batch size
dataset_name = "BMI Datset"
learning_rate_preset = 0.01
batch_size_preset = 10

**Epsilon Pascal Challenge Dataset**

In [ ]:
### Epsilon Pascal Challenge Dataset

# Loading training data
dataset = pd.read_csv("epsilon_normalized.txt", sep=' ', header=None, nrows=20000)
dataset = dataset.to_numpy()
for i in range(1, dataset.shape[1]-1):
    dataset[:, i] = [float(value.split(':')[1]) if isinstance(value, str) else value for value in dataset[:, i]]
dataset = dataset[:, :-1]
np.random.shuffle(dataset)

for i in range(1, dataset.shape[0]):
    if dataset[i - 1, 0] == -1:
        dataset[i - 1, 0] = 0

# Splitting off data for training and validation
train_split = int(0.8 * len(dataset))
X_train = dataset[:train_split, 1:].astype(np.float32)
y_train = dataset[:train_split, 0].astype(np.float32)
X_test = dataset[train_split:, 1:].astype(np.float32)
y_test = dataset[train_split:, 0].astype(np.float32)
#print(X_train)
#print(y_train)

# Normalize data set
X_train_normalized = X_train
X_test_normalized = X_test

# Turn data to tensor
X_train_tensor = torch.from_numpy(X_train_normalized)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test_normalized)
y_test_tensor = torch.from_numpy(y_test)
print("X_train_tensor size: ", X_train_tensor.size())
print("y_train_tensor size: ", y_train_tensor.size())
print("X_test_tensor size: ", X_test_tensor.size())
print("y_test_tensor size: ", y_test_tensor.size())

# Preset parameters
dataset_name = "Epsilon Datset"
learning_rate_preset = 0.001
batch_size_preset = 10

**Gisette Dataset**

In [ ]:
### gisette Dataset
#2 class, 6000 data points, ~5000 features

# Loading training data
dataset = pd.read_csv("gisette_scale", sep=' ', header=None)
dataset = dataset.to_numpy()
for i in range(1, dataset.shape[1]-1):
    dataset[:, i] = [float(value.split(':')[1]) if isinstance(value, str) else value for value in dataset[:, i]]
dataset = dataset[:, :-2]
np.random.shuffle(dataset)

for i in range(1, dataset.shape[0]):
    if dataset[i - 1, 0] == -1:
        dataset[i - 1, 0] = 0

#print(dataset)
#print(dataset.shape)

# Splitting off data for training and validation
train_split = int(0.8 * len(dataset))
X_train = dataset[:train_split, 1:].astype(np.float32)
y_train = dataset[:train_split, 0].astype(np.float32)
X_test = dataset[train_split:, 1:].astype(np.float32)
y_test = dataset[train_split:, 0].astype(np.float32)
print(X_train)
print(y_train)

# Normalize data set
denominator = X_train.max(0) - X_train.min(0)
X_train_normalized = (X_train - X_train.min(0)) / np.where(denominator != 0, denominator, 1)
denominator = X_test.max(0) - X_test.min(0)
X_test_normalized = (X_test - X_test.min(0)) / np.where(denominator != 0, denominator, 1)

print("X_train_normalized: ", X_train_normalized)
print("X_test_normalized: ", X_test_normalized)

# Turn data to tensor
X_train_tensor = torch.from_numpy(X_train_normalized)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test_normalized)
y_test_tensor = torch.from_numpy(y_test)
print(X_train_tensor.size())
print(y_train_tensor.size())
print(X_test_tensor.size())
print(y_test_tensor.size())

# Learning Rate and Batch size
dataset_name = "Gisette Datset"
learning_rate_preset = 0.005
batch_size_preset = 100

### Section 1.1. Vanilia Gradient Descent and Stochastic Gradient Descent

Initially test training dataset with custom algorithms and pytorch package. We will verify our result of our algorithms by comparing the pytorch package.

**Custom Algorithm for Vanilia Gradient Descent**

In [ ]:
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Vanilia Gradient Descent Algorithms
def gradient_descent(X, y, learning_rate, num_epochs):
    num_samples, num_features = X.shape
    
    # Initialize weights and bias
    w = np.zeros(num_features)
    b = 0

    loss_history = []
    error_history = []
    
    for epoch in range(num_epochs):
        # Calculate predictions
        y_pred = np.dot(X, w) + b
        
        # Calculate the difference between predictions and actual values
        error = y_pred - y
        
        # Calculate the gradient
        w_gradient = (1/num_samples) * np.dot(X.T, error)
        b_gradient = (1/num_samples) * np.sum(error)
        
        # Update theta using the learning rate and gradient
        w -= learning_rate * w_gradient
        b -= learning_rate * b_gradient

        # Record the loss
        loss = np.mean(np.square(error))
        loss_history.append(loss)

        # Record the error rate
        train_error_rate = calculate_error_rate(X_train_normalized,  y_train, torch.from_numpy(y_pred))
        error_history.append(train_error_rate)

        # Print the loss every specific epochs
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')
    
    return w, b, loss_history, error_history

# Train the model using gradient descent
learning_rate = learning_rate_preset
num_iterations = 100
w, b, loss_history, error_history = gradient_descent(X_train_normalized, y_train, learning_rate, num_iterations)

# Print the learned parameters
print("Learned parameters:")

for i, w_i in enumerate(w):
    print(f"w{i} =", w_i)
print("b =", b)

# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
y_pred = np.dot(X_train_normalized, w) + b
train_error_rate = calculate_error_rate(X_train_normalized,  y_train, torch.from_numpy(y_pred))
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    y_pred = np.dot(X_test_normalized, w) + b
    test_error_rate = calculate_error_rate(X_test_normalized,  y_test, torch.from_numpy(y_pred))
    print("Test error rate:", test_error_rate)

**Custom Algorithm for Stochastic Gradient Descent**

In [ ]:
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Stochastic Gradient Descent Algorithms
def stochastic_gradient_descent(X, y, learning_rate, num_epochs, batch_size):
    num_samples, num_features = X.shape
    num_batches = num_samples // batch_size

    # Initialize weights and bias
    w = np.zeros(num_features)
    b = 0
    
    loss_history = []
    error_history = []

    for epoch in range(num_epochs):
        # Shuffle the data for each epoch
        permutation = np.random.permutation(num_samples)
        X_shuffled = X[permutation]
        y_shuffled = y[permutation]

        for batch in range(num_batches):
            # Select the current batch
            start = batch * batch_size
            end = (batch + 1) * batch_size
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]

            # Calculate predictions
            y_pred = np.dot(X_batch, w) + b

            # Calculate the difference between predictions and actual values
            error = y_pred - y_batch

            # Calculate the gradients
            w_gradient = (1 / batch_size) * np.dot(X_batch.T, error)
            b_gradient = (1 / batch_size) * np.sum(error)

            # Update weights and bias
            w -= learning_rate * w_gradient
            b -= learning_rate * b_gradient
        
        # General Output
        y_pred = np.dot(X_train_normalized, w) + b
        error = y_pred - y_train

        # Record the loss
        error = y_pred
        loss = np.mean(np.square(error))
        loss_history.append(loss)

        # Record the error rate
        train_error_rate = calculate_error_rate(X_train_normalized,  y_train, torch.from_numpy(y_pred))
        error_history.append(train_error_rate)

        # Print the loss every specific epochs
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')
            
    return w, b, loss_history, error_history

# Train the model using stochastic gradient descent
learning_rate = learning_rate_preset
num_epochs = 1000
batch_size = batch_size_preset
w, b, loss_history, error_history = stochastic_gradient_descent(X_train_normalized, y_train, learning_rate, num_epochs, batch_size)

# Print the learned parameters
print("Learned parameters:")
for i, w_i in enumerate(w):
    print(f"w{i} =", w_i)
print("b =", b)

# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
y_pred = np.dot(X_train_normalized, w) + b
train_error_rate = calculate_error_rate(X_train_normalized,  y_train, torch.from_numpy(y_pred))
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    y_pred = np.dot(X_test_normalized, w) + b
    test_error_rate = calculate_error_rate(X_test_normalized,  y_test, torch.from_numpy(y_pred))
    print("Test error rate:", test_error_rate)

**Pytorch Package for Vanilia Gradient Descent**

(It is just a Vanilia Gradient Descent... They call it SGD is confusing...)

In [ ]:
### Pytorch Vanilia Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 1000

# Define the number of features
num_features = X_train_tensor.size()[1]

# Define the model parameters (weights and bias)
w = torch.zeros(num_features, dtype=torch.float, requires_grad=True)
# w = torch.tensor([1., 1., 1.], requires_grad=True)
b = torch.zeros(1, dtype=torch.float, requires_grad=True)
# b = torch.tensor([1.], requires_grad=True)

loss_history = []
error_history = []

# Define the loss function
criterion = torch.nn.MSELoss()

# Define the optimizer (Vanilla Gradient Descent)
optimizer = torch.optim.SGD([w, b], lr=learning_rate, weight_decay=0)

# Perform gradient descent
for epoch in range(num_epochs):
    # Forward pass
    outputs = torch.matmul(X_train_tensor.float(), w) + b
    loss = criterion(outputs, y_train_tensor.float())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record the loss
    loss_history.append(loss.detach().numpy())
    
    # Record the error rate
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    error_history.append(train_error_rate)

    # Print the loss every specific epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')
        

# Print learned parameters
print('Trained weights:', w)
print('Trained bias:', b)

# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
outputs = torch.matmul(X_train_tensor.float(), w) + b
train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    outputs = torch.matmul(X_test_tensor.float(), w) + b
    test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
    print("Test error rate:", test_error_rate)

**Pytorch Package for Stochastic Gradient Descent**

In [ ]:
### Pytorch Stochastic Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs and batch size
learning_rate = learning_rate_preset
num_epochs = 500
batch_size = batch_size_preset

# Define the number of samples and features
num_samples  = X_train_tensor.size()[0]
num_features = X_train_tensor.size()[1]

# Define the model parameters (weights and bias)
w = torch.zeros(num_features, dtype=torch.float, requires_grad=True)
# w = torch.tensor([1., 1., 1.], requires_grad=True)
b = torch.zeros(1, dtype=torch.float, requires_grad=True)
# b = torch.tensor([1.], requires_grad=True)

loss_history = []
error_history = []

# Define the loss function
criterion = torch.nn.MSELoss()

# Define the optimizer (Vanilla Gradient Descent)
optimizer = torch.optim.SGD([w, b], lr=learning_rate, weight_decay=0)

# Perform stochastic gradient descent
num_batches = num_samples // batch_size
for epoch in range(num_epochs):
    # Shuffle the data for each epoch
    permutation = torch.randperm(num_samples)
    X_shuffled = X_train_tensor[permutation]
    y_shuffled = y_train_tensor[permutation]
    for batch in range(num_batches):
        # Select the current batch
        start = batch * batch_size
        end = (batch + 1) * batch_size
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]

        # Forward pass
        outputs = torch.matmul(X_batch.float(), w) + b
        loss = criterion(outputs, y_batch.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # General Output
    outputs = torch.matmul(X_train_tensor.float(), w) + b
    loss = criterion(outputs, y_train_tensor.float())

    # Record the loss
    loss_history.append(loss.detach().numpy())
    
    # Record the error rate
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    error_history.append(train_error_rate)

    # Print the loss every specific epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')

# Print learned parameters
print('Trained weights:', w)
print('Trained bias:', b)

# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
outputs = torch.matmul(X_train_tensor.float(), w) + b
train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    outputs = torch.matmul(X_test_tensor.float(), w) + b
    test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
    print("Test error rate:", test_error_rate)

### Section 1.2. Linear Neural Network

**Neural Network**

This neural network is simply a linear model.

In [ ]:
# Define a custom neural network class
class NeuralNetwork(nn.Module):
    def __init__(self, num_features=1):
        super().__init__()
        self.flatten = nn.Flatten()
        self.activation_stack = nn.Sequential(
            nn.Linear(num_features, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.activation_stack(x)
        return torch.squeeze(logits)

**Linear Neural Network with Vanilia Gradient Descent**

In [ ]:
### Neural Network with Vanilia Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 1000

# Define the number of samples and features
num_samples  = X_train_tensor.size()[0]
num_features = X_train_tensor.size()[1]
num_class = len(torch.unique(y_train_tensor))
print(num_samples)
print(num_features)
print(num_class)

# Define the model parameters
loss_history = []
error_history = []

# Define neural network model, loss criterion and optimizer
criterion = torch.nn.MSELoss()
model = NeuralNetwork(num_features)
print(model)
optimizer = custom_optimizer_SGD(model.parameters(), lr=learning_rate, weight_decay = 0)

#for name, param in NeuralNetwork_model.named_parameters():
#    print( name )
#    values = torch.ones( param.shape )
#    param.data = values
    
# Perform training
model.train()
for epoch in range(num_epochs):
    # Forward propagation to obtain the predicted output
    outputs = model(X_train_tensor.float())
    
    # Calculate the loss
    loss = criterion(outputs, y_train_tensor.float())
    
    # Backward propagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Record the loss
    loss_history.append(loss.detach().numpy())
    
    # Record the error rate
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    error_history.append(train_error_rate)
    
    # Print the loss every specific epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')
        
# Print learned parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.data}')
        
# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
outputs = model(X_train_tensor.float())
train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    outputs = model(X_test_tensor.float())
    test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
    print("Test error rate:", test_error_rate)

**Linear Neural Network with Stochastic Gradient Descent**

In [ ]:
### Neural Network with Stochastic Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs and batch size
learning_rate = learning_rate_preset
num_epochs = 1000
batch_size = batch_size_preset

# Define the number of samples and features
num_samples  = X_train_tensor.size()[0]
num_features = X_train_tensor.size()[1]

# Define the model parameters
loss_history = []
error_history = []

# Define neural network model, loss criterion and optimizer
criterion = torch.nn.MSELoss()
model = NeuralNetwork(num_features)
print(model)
optimizer = custom_optimizer_SGD(model.parameters(), lr=learning_rate, weight_decay = 0)

#for name, param in NeuralNetwork_model.named_parameters():
#    print( name )
#    values = torch.ones( param.shape )
#    param.data = values
    
# Perform training
model.train()
num_batches = num_samples // batch_size
for epoch in range(num_epochs):
    # Shuffle the data for each epoch
    permutation = torch.randperm(num_samples)
    X_shuffled = X_train_tensor[permutation]
    y_shuffled = y_train_tensor[permutation]
    for batch in range(num_batches):
        # Select the current batch
        start = batch * batch_size
        end = (batch + 1) * batch_size
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]

        # Forward propagation to obtain the predicted output
        outputs = model(X_batch.float())
    
        # Calculate the loss
        loss = criterion(outputs, y_batch.float())
    
        # Backward propagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # General Output
    outputs = model(X_train_tensor.float())
    loss = criterion(outputs, y_train_tensor.float())

    # Record the loss
    loss_history.append(loss.detach().numpy())
    
    # Record the error rate
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    error_history.append(train_error_rate)

    # Print the loss every specific epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}')
        
# Print learned parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.data}')
        
# Plot the training loss history
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss History")
plt.show()

# Plot the error rate history
plt.plot(error_history)
plt.xlabel("Iteration")
plt.ylabel("Error Rate")
plt.title("Error Rate History")
plt.show()

# Calculate train error rate
outputs = model(X_train_tensor.float())
train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
print("Train error rate:", train_error_rate)
    
# Calculate test error rate if test data is provided
if X_test is not None and y_test is not None:
    outputs = model(X_test_tensor.float())
    test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
    print("Test error rate:", test_error_rate)

### Section 1.3. Fedearted Learning

#### Section 1.3.1 Fedearted Learning Training

**Training function for Federated Learning with Vanilia Gradient Descent**

In [ ]:
### Federated Learning Training with vanilia gradient descent method
def FedLearnTrainGDVanilia(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total):
    # Define neural network model, loss criterion and optimizer
    model = NeuralNetwork(X_train_tensor.size()[1])
    criterion = torch.nn.MSELoss()
    optimizer = custom_optimizer_SGD(model.parameters(), lr=learning_rate)

    # Preprocess the client data
    X_train_client = [None] * num_clients
    y_train_client = [None] * num_clients
    client_row = math.floor( X_train_tensor.size(dim=0) / num_clients )
    for client in range(num_clients):
        X_train_client[client] = X_train_tensor[(client)*client_row : (client+1)*client_row]
        y_train_client[client] = y_train_tensor[(client)*client_row : (client+1)*client_row]

    # Establish client devices
    client_model = [None] * num_clients
    client_optimizer = [None] * num_clients
    client_device = [None] * num_clients
    client_weights = [None] * num_clients
    for client in range(num_clients):
        client_model[client] = NeuralNetwork(X_train_client[client].size()[1])
        client_optimizer[client] = custom_optimizer_SGD(client_model[client].parameters(), lr=learning_rate)
        client_device[client] = ClientDevice(client_model[client], criterion, client_optimizer[client], X_train_client[client], y_train_client[client])

    # Cost History
    loss_cost_history = []
    error_cost_history = []
    send_cost_history = []
    send_cost = 0

    # Perform training
    global_weights = model.state_dict()
    #print(f'Initial global weights are: {global_weights}')
    for epoch in range(num_epochs):
        client_weights_total = []

        # Clients local update
        for client in range(num_clients):
            # Transmit the global weight to clients
            client_device[client].load_global_weights(global_weights)
            client_weights[client] = client_device[client].update_weights_GDVanilia(local_update_epochs)

            # Send client weights to the server
            send_client_weights(client_weights_total, client_weights[client])
            client_weights_size = sum(len(value) for value in client_weights[client].values())
            send_cost = send_cost + client_weights_size

        # Aggregate client weights on the server
        aggregated_weights = Federated_Averaging(client_weights_total)

        # Update global weights with aggregated weights
        global_weights.update(aggregated_weights)
        model.load_state_dict(global_weights)

        # General Output
        outputs = model(X_train_tensor.float())
        loss = criterion(outputs, y_train_tensor.float())

        # Record the loss
        loss_cost_history.append(loss.item())

        # Record the error rate
        train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
        error_cost_history.append(train_error_rate)

        # Record the send cost
        send_cost_history.append(send_cost)

        # Print the loss every specific epochs
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}, Culminative Send Cost: {send_cost}')

    # Print learned parameters
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'{name}: {param.data}')

    # Plot send cost history
    plt.plot(send_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Send Cost")
    plt.title("Send Cost History")
    plt.show()
    print(f'Total send cost: {send_cost}')

    # Plot the training loss history
    plt.plot(loss_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Training Loss Rate")
    plt.title("Training Loss History")
    plt.show()

    # Plot the error rate history
    plt.plot(error_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Error Rate")
    plt.title("Error Rate History")
    plt.show()

    # Calculate train error rate
    outputs = model(X_train_tensor.float())
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    print("Train error rate:", train_error_rate)
        
    # Calculate test error rate if test data is provided
    if X_test is not None and y_test is not None:
        outputs = model(X_test_tensor.float())
        test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
        print("Test error rate:", test_error_rate)

    # Record the history of loss, error and send_cost
    loss_cost_history_total.append(loss_cost_history)
    error_cost_history_total.append(error_cost_history)
    send_cost_history_total.append(send_cost_history)

**Training function for Federated Learning with Stochastic Gradient Descent**

In [ ]:
### Federated Learning Training with stochastic gradient descent method
def FedLearnTrainGDStochastic(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total):
    # Define neural network model, loss criterion and optimizer
    model = NeuralNetwork(X_train_tensor.size()[1])
    criterion = torch.nn.MSELoss()
    optimizer = custom_optimizer_SGD(model.parameters(), lr=learning_rate)

    # Preprocess the client data
    X_train_client = [None] * num_clients
    y_train_client = [None] * num_clients
    client_row = math.floor( X_train_tensor.size(dim=0) / num_clients )
    for client in range(num_clients):
        X_train_client[client] = X_train_tensor[(client)*client_row : (client+1)*client_row]
        y_train_client[client] = y_train_tensor[(client)*client_row : (client+1)*client_row]

    # Establish client devices
    client_model = [None] * num_clients
    client_optimizer = [None] * num_clients
    client_device = [None] * num_clients
    client_weights = [None] * num_clients
    for client in range(num_clients):
        client_model[client] = NeuralNetwork(X_train_client[client].size()[1])
        client_optimizer[client] = custom_optimizer_SGD(client_model[client].parameters(), lr=learning_rate)
        client_device[client] = ClientDevice(client_model[client], criterion, client_optimizer[client], X_train_client[client], y_train_client[client])

    # Cost History
    loss_cost_history = []
    error_cost_history = []
    send_cost_history = []
    send_cost = 0

    # Perform training
    global_weights = model.state_dict()
    #print(f'Initial global weights are: {global_weights}')
    for epoch in range(num_epochs):
        client_weights_total = []

        # Clients local update
        for client in range(num_clients):
            # Transmit the global weight to clients
            client_device[client].load_global_weights(global_weights)
            client_weights[client] = client_device[client].update_weights_GDStochastic(local_update_epochs, int(batch_size / num_clients))

            # Send client weights to the server
            send_client_weights(client_weights_total, client_weights[client])
            client_weights_size = sum(len(value) for value in client_weights[client].values())
            send_cost = send_cost + client_weights_size

        # Aggregate client weights on the server
        aggregated_weights = Federated_Averaging(client_weights_total)

        # Update global weights with aggregated weights
        global_weights.update(aggregated_weights)
        model.load_state_dict(global_weights)

        # General Output
        outputs = model(X_train_tensor.float())
        loss = criterion(outputs, y_train_tensor.float())

        # Record the loss
        loss_cost_history.append(loss.item())

        # Record the error rate
        train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
        error_cost_history.append(train_error_rate)

        # Record the send cost
        send_cost_history.append(send_cost)

        # Print the loss every specific epochs
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}, Error: {train_error_rate:.16f}, Culminative Send Cost: {send_cost}')

    # Print learned parameters
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'{name}: {param.data}')

    # Plot send cost history
    plt.plot(send_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Send Cost")
    plt.title("Send Cost History")
    plt.show()
    print(f'Total send cost: {send_cost}')

    # Plot the training loss history
    plt.plot(loss_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Training Loss Rate")
    plt.title("Training Loss History")
    plt.show()

    # Plot the error rate history
    plt.plot(error_cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Error Rate")
    plt.title("Error Rate History")
    plt.show()

    # Calculate train error rate
    outputs = model(X_train_tensor.float())
    train_error_rate = calculate_error_rate(X_train_tensor,  y_train_tensor, outputs)
    print("Train error rate:", train_error_rate)
        
    # Calculate test error rate if test data is provided
    if X_test is not None and y_test is not None:
        outputs = model(X_test_tensor.float())
        test_error_rate = calculate_error_rate(X_test_tensor,  y_test_tensor, outputs)
        print("Test error rate:", test_error_rate)

    # Record the history of loss, error and send_cost
    loss_cost_history_total.append(loss_cost_history)
    error_cost_history_total.append(error_cost_history)
    send_cost_history_total.append(send_cost_history)

#### Section 1.3.2 Fedearted Learning Experiment

**Experiment with number of clients of Vanilia Gradient Descent**

In [ ]:
### Experiment for num_clients with Vanilia Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 500
num_clients_list = [1,5,10,15,20]
local_update_epochs_list = [1]

# Cost History Total
num_clients_list_size = len(num_clients_list)
local_update_epochs_list_size = len(local_update_epochs_list)
loss_cost_history_total = []
error_cost_history_total = []
send_cost_history_total = []

# Compare the num_clients
local_update_epochs = local_update_epochs_list[0]
for num_clients in num_clients_list:
    print(f'=== The training for num_clients is {num_clients} ===')
    FedLearnTrainGDVanilia(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total)

print(f'=== The Experiment Result ===')
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
print("Vanilia Gradient Descent")

# Plot the training loss rate between cost history with clients
for i in range(num_clients_list_size):
    plt.plot(send_cost_history_total[i], loss_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost (with different clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'Loss_VS_SendCost_num_clients_{dataset_name}_VGD.png')

# Plot the error rate between cost history with clients
for i in range(num_clients_list_size):
    plt.plot(send_cost_history_total[i], error_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost (with different clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'ErrorRate_VS_SendCost_num_clients_{dataset_name}_VGD.png')

# Plot the training loss rate between cost history with num_clients per clients
min_cost_list = send_cost_history_total[0]
for sublist in send_cost_history_total:
    if len(sublist) < len(min_cost_list):
        min_cost_list = sublist
for i in range(num_clients_list_size):
    plt.plot(min_cost_list, loss_cost_history_total[i])
plt.xlabel("Send Cost per clients")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost per clients (with different number of clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'Loss_vs_Send_Cost_num_clients_{dataset_name}_VGD_per_clients.png')

# Plot the training loss rate between cost history with num_clients per clients
min_cost_list = send_cost_history_total[0]
for sublist in send_cost_history_total:
    if len(sublist) < len(min_cost_list):
        min_cost_list = sublist
for i in range(num_clients_list_size):
    plt.plot(min_cost_list, error_cost_history_total[i])
plt.xlabel("Send Cost per clients")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost per clients (with different number of clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'ErrorRate_vs_Send_Cost_num_clients_{dataset_name}_VGD_per_clients.png')

**Experiment with number of clients of Stochastic Gradient Descent**

In [ ]:
### Experiment for num_clients with Stochastic Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 100
batch_size = batch_size_preset
num_clients_list = [1,5,10,15,20]
local_update_epochs_list = [2]

# Cost History Total
num_clients_list_size = len(num_clients_list)
local_update_epochs_list_size = len(local_update_epochs_list)
loss_cost_history_total = []
error_cost_history_total = []
send_cost_history_total = []

# Compare the num_clients
local_update_epochs = local_update_epochs_list[0]
for num_clients in num_clients_list:
    print(f'=== The training for num_clients is {num_clients} ===')
    FedLearnTrainGDStochastic(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total)

print(f'=== The Experiment Result ===')
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
print("Stochastic Gradient Descent")

# Plot the training loss rate between cost history with clients
for i in range(num_clients_list_size):
    plt.plot(send_cost_history_total[i], loss_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost (with different clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'Loss_VS_SendCost_num_clients_{dataset_name}_SGD.png')

# Plot the error rate between cost history with clients
for i in range(num_clients_list_size):
    plt.plot(send_cost_history_total[i], error_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost (with different clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'ErrorRate_VS_SendCost_num_clients_{dataset_name}_SGD.png')

# Plot the training loss rate between cost history with num_clients per clients
min_cost_list = send_cost_history_total[0]
for sublist in send_cost_history_total:
    if len(sublist) < len(min_cost_list):
        min_cost_list = sublist
for i in range(num_clients_list_size):
    plt.plot(min_cost_list, loss_cost_history_total[i])
plt.xlabel("Send Cost per clients")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost per clients (with different number of clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'Loss_vs_Send_Cost_num_clients_{dataset_name}_SGD_per_clients.png')

# Plot the training loss rate between cost history with num_clients per clients
min_cost_list = send_cost_history_total[0]
for sublist in send_cost_history_total:
    if len(sublist) < len(min_cost_list):
        min_cost_list = sublist
for i in range(num_clients_list_size):
    plt.plot(min_cost_list, error_cost_history_total[i])
plt.xlabel("Send Cost per clients")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost per clients (with different number of clients)")
plt.legend(num_clients_list)
plt.show()
plt.savefig(f'ErrorRate_vs_Send_Cost_num_clients_{dataset_name}_SGD_per_clients.png')

**Experiment with local update epochs of Vanilia Gradient Descent**

In [ ]:
### Experiment for local_update_epochs with Vanilia Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 1000
num_clients_list = [5]
local_update_epochs_list = [1,2,3,4,5]

# Cost History Total
num_clients_list_size = len(num_clients_list)
local_update_epochs_list_size = len(local_update_epochs_list)
loss_cost_history_total = []
error_cost_history_total = []
send_cost_history_total = []

# Compare the local_update_epochs
num_clients = num_clients_list[0]
for local_update_epochs in local_update_epochs_list:
    print(f'=== The training for local_update_epochs is {local_update_epochs} ===')
    FedLearnTrainGDVanilia(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total)

print(f'=== The Experiment Result ===')
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
print("Vanilia Gradient Descent")

# Plot the training loss rate between cost history with local update epochs
for i in range(local_update_epochs_list_size):
    plt.plot(send_cost_history_total[i], loss_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost (with different local update epochs)")
plt.legend(local_update_epochs_list)
plt.show()
plt.savefig(f'Loss_VS_SendCost_local_update_epochs_{dataset_name}_VGD.png')

# Plot the error rate between cost history with local update epochs
for i in range(local_update_epochs_list_size):
    plt.plot(send_cost_history_total[i], error_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost (with different local update epochs)")
plt.legend(local_update_epochs_list)
plt.show()
plt.savefig(f'ErrorRate_VS_SendCost_local_update_epochs_{dataset_name}_VGD.png')

**Experiment with local update epochs of Stochastic Gradient Descent**

In [ ]:
### Experiment for local_update_epochs with Stochastic Gradient Descent

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 500
batch_size = batch_size_preset
num_clients_list = [2]
local_update_epochs_list = [1,2,3,4,5]

# Cost History Total
num_clients_list_size = len(num_clients_list)
local_update_epochs_list_size = len(local_update_epochs_list)
loss_cost_history_total = []
error_cost_history_total = []
send_cost_history_total = []

# Compare the local_update_epochs
num_clients = num_clients_list[0]
for local_update_epochs in local_update_epochs_list:
    print(f'=== The training for local_update_epochs is {local_update_epochs} ===')
    FedLearnTrainGDStochastic(num_clients, local_update_epochs, loss_cost_history_total, error_cost_history_total, send_cost_history_total)

print(f'=== The Experiment Result ===')
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
print("Stochastic Gradient Descent")

# Plot the training loss rate between cost history with local update epochs
for i in range(local_update_epochs_list_size):
    plt.plot(send_cost_history_total[i], loss_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Training Loss Rate")
plt.title("Training Loss Rate vs Send Cost (with different local update epochs)")
plt.legend(local_update_epochs_list)
plt.show()
plt.savefig(f'Loss_VS_SendCost_local_update_epochs_{dataset_name}_SGD.png')

# Plot the error rate between cost history with local update epochs
for i in range(local_update_epochs_list_size):
    plt.plot(send_cost_history_total[i], error_cost_history_total[i])
plt.xlabel("Send Cost")
plt.ylabel("Error Rate")
plt.title("Error Rate vs Send Cost (with different local update epochs)")
plt.legend(local_update_epochs_list)
plt.show()
plt.savefig(f'ErrorRate_VS_SendCost_local_update_epochs_{dataset_name}_SGD.png')

## Section 2 Non-Linear Training Model
Here we focus on dataset requires mainly non-linear training algorithms.

### Section 2.0 Data Loading and Preprocessing

Here we load the data for the expriements.

**MNIST Dataset**

In [ ]:
### MNIST Dataset

# Loading training data
dataset = pd.read_csv("mnist_train.csv")
dataset = dataset.to_numpy()
X_train = dataset[:10000, 1:].astype(np.int32)
y_train = dataset[:10000, 0].astype(np.int32)

# Loading testing data
dataset = pd.read_csv("mnist_test.csv")
dataset = dataset.to_numpy()
X_test = dataset[:1000, 1:].astype(np.int32)
y_test = dataset[:1000, 0].astype(np.int32)

# Translation of data  
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
y_train = y_train.astype('float32') 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
y_test = y_test.astype('float32') 
print(X_train)
print(y_train)

# Normalize data set
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

# Turn data to tensor
X_train_tensor = torch.from_numpy(X_train_normalized)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test_normalized)
y_test_tensor = torch.from_numpy(y_test)
print(X_train_tensor)
print(y_train_tensor)
print(X_test_tensor)
print(y_test_tensor)
print(X_train_tensor.size())
print(y_train_tensor.size())
print(X_test_tensor.size())
print(y_test_tensor.size())

# Learning Rate and Batch size
dataset_name = "MNIST Dataset"
learning_rate_preset = 0.01
batch_size_preset = 100
CNN_input_shape_preset = (-1, 1, 28, 28)

# Number of samples, features, and classes
num_samples_preset  = X_train_tensor.size()[0]
num_features_preset = X_train_tensor.size()[1]
num_classes_preset = len(torch.unique(y_train_tensor))

# Translate the tensor to dataset
MINST_train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
MINST_test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# Translate to DataLoader
train_loader = torch.utils.data.DataLoader(MINST_train_dataset, batch_size = batch_size_preset, shuffle = True)
test_loader = torch.utils.data.DataLoader(MINST_test_dataset, batch_size = batch_size_preset, shuffle = True)

# Visualize the first 10 images
for i in range(10):
    plt.imshow(X_train_tensor[i], cmap='gray')
    plt.show()
    print(y_train_tensor[i])

**CIFAR-10 Dataset**

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10_data(directory):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    
    # Load data batches
    for i in range(1, 6):
        train_batch_file = os.path.join(directory, 'data_batch_{}'.format(i))
        train_data_dict = unpickle(train_batch_file)
        train_batch_images = train_data_dict[b'data']
        train_batch_labels = train_data_dict[b'labels']
        
        # Reshape the image data
        train_batch_images = train_batch_images.reshape((-1, 3, 32, 32)).transpose((0, 2, 3, 1))
        
        train_images.append(train_batch_images)
        train_labels.extend(train_batch_labels)
    
    # Load test batch
    test_batch_file = os.path.join(directory, 'test_batch')
    test_data_dict = unpickle(test_batch_file)
    test_batch_images = test_data_dict[b'data']
    test_batch_labels = test_data_dict[b'labels']
    
    # Reshape the test image data
    test_batch_images = test_batch_images.reshape((-1, 3, 32, 32)).transpose((0, 2, 3, 1))

    test_images.append(test_batch_images)
    test_labels.extend(test_batch_labels)
    
    train_images = np.concatenate(train_images, axis=0)
    train_labels = np.array(train_labels)
    test_images = np.concatenate(test_images, axis=0)
    test_labels = np.array(test_labels)
    
    return train_images, train_labels, test_images, test_labels

# Specify the path to the 'cifar-10-batches-py' directory
data_directory = 'cifar-10-batches-py'

# Load CIFAR-10 data
train_images, train_labels, test_images, test_labels = load_cifar10_data(data_directory)

# Input to X and y
X_train = train_images.astype(np.int32)
y_train = train_labels.astype(np.int32)
X_test = test_images.astype(np.int32)
y_test = test_labels.astype(np.int32)

# Translation of data
X_train = X_train.astype('float32')
y_train = y_train.astype('float32') 
X_test = X_test.astype('float32')
y_test = y_test.astype('float32') 
print(X_train)
print(y_train)

# Normalize data set
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

# Turn data to tensor
X_train_tensor = torch.from_numpy(X_train_normalized[:100])
y_train_tensor = torch.from_numpy(y_train[:100])
X_test_tensor = torch.from_numpy(X_test_normalized)
y_test_tensor = torch.from_numpy(y_test)
print(X_train_tensor)
print(y_train_tensor)
print(X_test_tensor)
print(y_test_tensor)
print(X_train_tensor.size())
print(y_train_tensor.size())
print(X_test_tensor.size())
print(y_test_tensor.size())

# Learning Rate and Batch size
dataset_name = "CIFAR-10 Datset"
learning_rate_preset = 0.01
batch_size_preset = 100
CNN_input_shape_preset = (-1, 3, 32, 32)

# Number of samples, features, and classes
num_samples_preset  = X_train_tensor.size()[0]
num_features_preset = X_train_tensor.size()[1]
num_classes_preset = len(torch.unique(y_train_tensor))

# Translate the tensor to dataset
CIFAR_10_train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
CIFAR_10_test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# Translate to DataLoader
train_loader = torch.utils.data.DataLoader(CIFAR_10_train_dataset, batch_size = batch_size_preset, shuffle = True)
test_loader = torch.utils.data.DataLoader(CIFAR_10_test_dataset, batch_size = batch_size_preset, shuffle = True)

# Visualize the first 10 images
for i in range(10):
    plt.imshow(X_train_tensor[i])
    plt.show()
    print(y_train_tensor[i])

### Section 2.1 Non-Linear Neural Networks

**Convolutional Neural Network (CNN)**

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 7 * 7, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return torch.squeeze(x)
    
class ConvolutionalNeuralNetwork_CIFAR10(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.activation_stack = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes))
        
    def forward(self, x):
        x = self.activation_stack(x)
        return torch.squeeze(x)

**Client Device for CNN**

In [ ]:
# Define a custom class for each client so they can update separately
class ClientDevice:
    def __init__(self, model, optimizer, criterion, train_dataloader, test_dataloader=None):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader

    def load_global_weights(self, global_weights):
        self.model.load_state_dict(global_weights)

    def get_local_weights(self):
        return self.model.state_dict()
    
    def update_weights(self, num_epochs, iterate_func):
        self.model.train()
        loss_history, error_history = iterate_func(self.model, self.train_dataloader, num_epochs, self.optimizer, self.criterion, show_history=False, training=True)
        return self.model.state_dict(), loss_history, error_history

# Establish client devices
def establish_client_devices(num_clients, train_dataloader_list, test_dataloader_list, model_list, optimizer_list, criterion_list):
    # Establish client devices
    client_device = [None] * num_clients
    client_model = [None] * num_clients
    client_optimizer = [None] * num_clients
    client_criterion = [None] * num_clients
    client_weights = [None] * num_clients
    for client in range(num_clients):
        client_model[client] = model_list[client]
        client_optimizer[client] = optimizer_list[client]
        client_criterion[client] = criterion_list[client]
        client_weights[client] = client_model[client].state_dict()
        client_device[client] = ClientDevice(client_model[client], client_optimizer[client], client_criterion[client], train_dataloader_list[client], test_dataloader_list[client])
    return client_device

**CNN Training and Testing Algorithm**

In [ ]:
def iterate_CNN_model(model, dataset_loader, num_epochs, optimizer, criterion, show_history=True, training=True):
    loss_history = []
    error_history = []
    for epoch in range(num_epochs):
        single_iteration_count = 0.00
        batch_iteration_count = 0.00
        loss_count = 0.00
        error_count = 0.00
        for i, (images, labels) in enumerate(dataset_loader):
            # Define variables
            X_batch = Variable(images.view(CNN_input_shape))
            y_batch = Variable(labels)

            # Forward propagation to obtain the predicted output
            outputs = model(X_batch)

            # Calculate the loss
            loss = criterion(outputs, y_batch.long())
            
            # Backward propagation and optimization
            if training is True:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Record the iteration used
            single_iteration_count += 1
            batch_iteration_count += len(y_batch)

            # Record the loss
            loss_count += loss.data.detach().numpy()

            # Record the error rate
            predicted = torch.max(outputs.data, 1)[1]
            error_count += (predicted != y_batch).float().sum()
        
        # Summarize the loss rate
        loss_rate = loss_count / float(single_iteration_count)
        loss_history.append(loss_rate)

        # Summarize the error rate
        train_error_rate = error_count / float(batch_iteration_count)
        error_history.append(train_error_rate)

        # Print the summarized loss and error every specific epochs
        if show_history is True and ((epoch + 1) % 10 == 0 or epoch == 0):
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {loss_rate.item():.8f}, Average Error: {train_error_rate:.16f}')

    return loss_history, error_history

def fit_CNN_model(model, train_loader, test_loader, num_epochs, optimizer, criterion, show_history=True):
    # Model becomes "Train Mode"
    model.train()
    if test_loader is not None:
       print("!-- Training Session --!")
    train_loss_history, train_error_history = iterate_CNN_model(model, train_loader, num_epochs, optimizer, criterion, show_history=show_history, training=True)
    
    if test_loader is not None:
        # Model becomes "Eval Mode"
        model.eval()
        print("!-- Testing Session --!")
        test_loss_history, test_error_history = iterate_CNN_model(model, test_loader, num_epochs, optimizer, criterion, show_history=show_history, training=True)
    
    # Experiment Result
    print("!-- CNN Model Result --!")

    # Print learned parameters
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'{name}: {param.data}')
            
    # Plot the training and testing loss history
    if test_loader is not None:
        plt.plot(train_loss_history, label='Training Loss')
        plt.plot(test_loss_history, label='Testing Loss')
        plt.legend()
    else:
        plt.plot(train_loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Average Training Loss Rate")
    plt.title("Average Training Loss History")
    plt.show()

    # Plot the error rate history
    if test_loader is not None:
        plt.plot(train_error_history, label='Training Error Rate')
        plt.plot(test_error_history, label='Testing Error Rate')
        plt.legend()
    else:
        plt.plot(train_error_history)
    plt.xlabel("Epochs")
    plt.ylabel("Average Error Rate")
    plt.title("Average Error Rate History")
    plt.show()

**Train the CNN model**

In [ ]:
# Show Dataset Name
print(f'Current Dataset: {dataset_name}')

# Define the learning rate and number of epochs
learning_rate = learning_rate_preset
num_epochs = 10

# Define neural network model, loss criterion and optimizer
criterion = nn.CrossEntropyLoss()
if dataset_name == "CIFAR-10 Datset":
    model = ConvolutionalNeuralNetwork_CIFAR10(num_classes_preset)
else:
    model = ConvolutionalNeuralNetwork(num_classes_preset)
CNN_input_shape = CNN_input_shape_preset
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0)
print(model)

# Perform training
start = timeit.default_timer()
fit_CNN_model(model, train_loader, None, num_epochs, optimizer, criterion, show_history=True)
stop = timeit.default_timer()
train_time = stop - start
print("Time: ", train_time)

### Section 2.2 Federated Learning Algorithms

In [ ]:
### Federated Learning Training with stochastic gradient descent method
def iterate_FedLearn_model(model, dataset_loader, num_epochs, optimizer, criterion, num_clients, local_update_epochs, client_device, iterate_func=iterate_CNN_model, aggregate_func=Federated_Averaging, show_history=True, training = True):
    # Initialize cost history for recording
    loss_cost_history = []
    error_cost_history = []
    send_cost_history = []
    time_history = []
    send_cost = 0

    # Initialize global weights
    client_weights = [None] * num_clients
    global_weights = model.state_dict()
    
    start_time = timeit.default_timer()
    # Iteration
    for epoch in range(num_epochs):
        client_weights_total = []
        loss_count = 0.00
        error_count = 0.00

        # Clients local update
        for client in range(num_clients):
            # Transmit the global weight to clients
            client_device[client].load_global_weights(global_weights)

            # Local update
            client_weights[client], local_loss_history, local_error_history = client_device[client].update_weights(local_update_epochs, iterate_func)
            local_loss = sum(local_loss_history) / len(local_loss_history)
            local_error = sum(local_error_history) / len(local_error_history)

            # Send client weights to the server
            send_client_weights(client_weights_total, client_weights[client])
            client_weights_size = sum(value.numel() for value in client_weights[client].values())
            send_cost = send_cost + client_weights_size

            # Send client loss and error to the server (we ignore the cost involved here)
            loss_count += local_loss
            error_count += local_error

        # Aggregate client weights on the server
        aggregated_weights = aggregate_func(client_weights_total)

        # Update global weights with aggregated weights
        global_weights.update(aggregated_weights)
        model.load_state_dict(global_weights)

        # Summarize loss and error
        loss_average = loss_count / num_clients
        train_error_rate_average = error_count / num_clients

        # Record the loss
        loss_cost_history.append(loss_average.item())

        # Record the error rate
        error_cost_history.append(train_error_rate_average)

        # Record the send cost
        send_cost_history.append(send_cost)

        # Record the time history
        current_time = timeit.default_timer()
        time_history.append(current_time)

        # Print the loss every specific epochs
        if show_history is True and ((epoch + 1) % 1 == 0 or epoch == 0):
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {loss_average.item():.8f}, Average Error: {train_error_rate_average:.16f}, Culminative Send Cost: {send_cost}')
    stop_time = timeit.default_timer()
    used_time = stop_time - start_time
    print("Time for all iteration: ", used_time)
    return loss_cost_history, error_cost_history, send_cost_history, time_history

def fit_FedLearn_model(model, train_loader, test_loader, num_epochs, num_clients, client_setup_func, local_update_epochs, optimizer, criterion, iterate_func, aggregate_func, show_history=True):
    # Preprocess the client data
    #client_dataloader_list = split_dataloader(train_loader, num_clients)

    # This is the alternative for preprocessing, will be changed in the future

    ### Alternative ###
    client_dataloader_list = [] * num_clients
    X_train_client = [None] * num_clients
    y_train_client = [None] * num_clients
    client_row = math.floor( num_samples_preset / num_clients )
    for client in range(num_clients):
        X_train_client[client] = X_train_tensor[(client)*client_row : (client+1)*client_row]
        y_train_client[client] = y_train_tensor[(client)*client_row : (client+1)*client_row]
        
        # Translate the tensor to dataset
        client_train_dataset = torch.utils.data.TensorDataset(X_train_client[client], y_train_client[client])

        # Translate to DataLoader
        client_loader = torch.utils.data.DataLoader(client_train_dataset, batch_size = batch_size_preset, shuffle = True)
        client_dataloader_list.append(client_loader)
    ### Alternative End ###

    # Define the client model, criterion, optimizer, its dataset
    client_model_list = [None] * num_clients
    client_optimizer_list = [None] * num_clients
    client_criterion_list = [None] * num_clients
    client_setup_func(num_clients, client_model_list, client_optimizer_list, client_criterion_list, client_dataloader_list)
    
    # Establish client devices
    client_device = establish_client_devices(num_clients=num_clients, 
                                             train_dataloader_list=client_dataloader_list, 
                                             test_dataloader_list= [None] * num_clients,
                                             model_list=client_model_list, 
                                             optimizer_list=client_optimizer_list,
                                             criterion_list=client_criterion_list)

    # Perform iteration
    train_loss_history = []
    train_error_history = []
    train_send_cost_history = []
    train_time_history = []
    test_loss_history = []
    test_error_history = []
    test_send_cost_history = []
    test_time_history = []
    
    # Model becomes "Train Mode"
    model.train()
    if test_loader is not None:
        print("!-- Training Session --!")
    train_loss_history, train_error_history, train_send_cost_history, train_time_history = iterate_FedLearn_model(model=model, dataset_loader=train_loader, 
                                                                                         num_epochs=num_epochs, optimizer=optimizer, criterion=criterion, 
                                                                                         num_clients=num_clients, local_update_epochs=local_update_epochs, client_device=client_device,
                                                                                         iterate_func=iterate_func, aggregate_func=aggregate_func,
                                                                                         show_history=show_history, training = True)

    if test_loader is not None:
        # Model becomes "Eval Mode"
        model.eval()
        print("!-- Testing Session --!")
        test_loss_history, test_error_history, test_send_cost_history, test_time_history = iterate_FedLearn_model(model=model, dataset_loader=test_loader, 
                                                                                         num_epochs=num_epochs, optimizer=optimizer, criterion=criterion, 
                                                                                         num_clients=num_clients, local_update_epochs=local_update_epochs, client_device=client_device,
                                                                                         iterate_func=iterate_func, aggregate_func=aggregate_func,
                                                                                         show_history=show_history, training = False)

    # Print learned parameters
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'{name}: {param.data}')
    
    # Plot send cost history
    if test_loader is not None:
        print(f'Total send cost in training: {sum(train_send_cost_history)}')
        print(f'Total send cost in testing: {sum(test_send_cost_history)}')
        plt.plot(train_send_cost_history, label='Training Send Cost')
        plt.plot(test_send_cost_history, label='Testing Send Cost')
        plt.legend()
    else:
        print(f'Total send cost in training: {sum(train_send_cost_history)}')
        plt.plot(train_send_cost_history)
    plt.plot(train_send_cost_history)
    plt.xlabel("Epochs")
    plt.ylabel("Send Cost")
    plt.title("Send Cost History")
    plt.show()
    
    # Plot the training and testing loss history
    if test_loader is not None:
        plt.plot(train_loss_history, label='Training Loss')
        plt.plot(test_loss_history, label='Testing Loss')
        plt.legend()
    else:
        plt.plot(train_loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Average Training Loss Rate")
    plt.title("Average Training Loss History")
    plt.show()

    # Plot the error rate history
    if test_loader is not None:
        plt.plot(train_error_history, label='Training Error Rate')
        plt.plot(test_error_history, label='Testing Error Rate')
        plt.legend()
    else:
        plt.plot(train_error_history)
    plt.xlabel("Epochs")
    plt.ylabel("Average Error Rate")
    plt.title("Average Error Rate History")
    plt.show()

    # Plot the train time history
    if test_loader is not None:
        plt.plot(train_time_history, label='Training Time History')
        plt.plot(test_time_history, label='Testing Time History')
        plt.legend()
    else:
        plt.plot(train_time_history)
    plt.xlabel("Epochs")
    plt.ylabel("Time used")
    plt.title("Iteration Time History")
    plt.show()

    return train_loss_history, train_error_history, train_send_cost_history, train_time_history, test_loss_history, test_error_history, test_send_cost_history, test_time_history

**Federated Learning Expriment Functions**

In [ ]:
def experiment_FedLearn_model(global_model_class_setup_func, train_loader, test_loader, num_clients_list, local_update_epochs_list, client_setup_func, iterate_func, aggregate_func, compareClients=False):
    # Cost History Total
    num_clients_list_size = len(num_clients_list)
    local_update_epochs_list_size = len(local_update_epochs_list)
    train_loss_history_total = []
    train_error_history_total = []
    train_send_cost_history_total = []
    train_time_history_total = []
    test_loss_history_total = []
    test_error_history_total = []
    test_send_cost_history_total = []
    test_time_history_total = []
    experiment_type = "test"

    if compareClients is True:
        iteration_list = num_clients_list
        experiment_type = "num_clients"
    else:
        iteration_list = local_update_epochs_list
        experiment_type = "local_update_epochs"

    for n in range(len(iteration_list)):
        if compareClients is True:
            print(f'=== The training for num_clients is {num_clients_list[n]} ===')
            num_clients = num_clients_list[n]
            local_update_epochs = local_update_epochs_list[0]
        else:
            print(f'=== The training for local_update_epochs is {local_update_epochs_list[n]} ===')
            num_clients = num_clients_list[0]
            local_update_epochs = local_update_epochs_list[n]

        # Define global neural network model, loss criterion and optimizer
        model, criterion, optimizer = global_model_class_setup_func()
        print(model)

        # Perform training
        train_loss_history, train_error_history, train_send_cost_history, train_time_history, test_loss_history, test_error_history, test_send_cost_history, test_time_history = fit_FedLearn_model(model, train_loader, test_loader, num_epochs, num_clients, client_setup_func, local_update_epochs, optimizer, criterion, iterate_func, aggregate_func, show_history=True)

        # Record the history of loss, error and send_cost
        train_loss_history_total.append(train_loss_history)
        train_error_history_total.append(train_error_history)
        train_send_cost_history_total.append(train_send_cost_history)
        train_time_history_total.append(train_time_history)
        test_loss_history_total.append(test_loss_history)
        test_error_history_total.append(test_error_history)
        test_send_cost_history_total.append(test_send_cost_history)
        test_time_history_total.append(test_time_history)

        # Save the npy result for client (train_loss, train_send_cost, train_time, test_error)
        #filename = "{}_{}_{}_{}epoch_result".format(dataset_name, experiment_type, iteration_list[n-1], num_epochs)
        #with open('filename', 'wb') as f:
        #    np.save(f, train_loss_history)
        #    np.save(f, train_error_history)
        #    np.save(f, train_send_cost_history)
        #    np.save(f, train_time_history)
        #    np.save(f, test_loss_history)
        #    np.save(f, test_error_history)
        #    np.save(f, test_send_cost_history)
        #    np.save(f, test_time_history)
    
    # Save the npy result (train_loss_history_total)
    total_filename = "{}_{}_{}epoch_result".format(dataset_name, experiment_type, num_epochs)
    with open(total_filename, 'wb') as f:
        np.savez(f, train_loss = train_loss_history_total, train_error = train_error_history_total, train_send_cost = train_send_cost_history_total, train_time = train_time_history_total, test_loss = test_loss_history_total, test_error = test_error_history_total, test_send_cost = test_send_cost_history, test_time = test_time_history_total)
        #np.save(f, )
        #np.save(f, )
        #np.save(f, )
        #np.save(f, )
        #np.save(f, )
        #np.save(f, )
        #np.save(f, )
    
    print(f'=== The Experiment Result ===')
    # Show Dataset Name
    print(f'Current Dataset: {dataset_name}')

    # Training Result
    print(f'!-- Training Result --!')
    if compareClients is True:
        # Plot the training loss rate between cost history with num_clients
        for i in range(num_clients_list_size):
            plt.plot(train_send_cost_history_total[i], train_loss_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_num_clients_training.png')

        # Plot the error rate between cost history with local_update_epochs
        for i in range(num_clients_list_size):
            plt.plot(train_send_cost_history_total[i], train_error_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_num_clients_training.png')

        # Plot the training loss rate between cost history with num_clients per clients
        min_cost_list = train_send_cost_history_total[0]
        for sublist in train_send_cost_history_total:
            if len(sublist) < len(min_cost_list):
                min_cost_list = sublist
        for i in range(num_clients_list_size):
            plt.plot(min_cost_list, train_loss_history_total[i])
        plt.xlabel("Send Cost per clients")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost per clients (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_num_clients_per_clients_training.png')

        # Plot the training loss rate between cost history with num_clients per clients
        min_cost_list = train_send_cost_history_total[0]
        for sublist in train_send_cost_history_total:
            if len(sublist) < len(min_cost_list):
                min_cost_list = sublist
        for i in range(num_clients_list_size):
            plt.plot(min_cost_list, train_error_history_total[i])
        plt.xlabel("Send Cost per clients")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost per clients (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_num_clients_per_clients_training.png')
    else:
        # Plot the training loss rate between cost history with local_update_epochs
        for i in range(local_update_epochs_list_size):
            plt.plot(train_send_cost_history_total[i], train_loss_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost (with different local update epochs)")
        plt.legend(local_update_epochs_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_local_update_epochs_training.png')

        # Plot the error rate between cost history with local_update_epochs
        for i in range(local_update_epochs_list_size):
            plt.plot(train_send_cost_history_total[i], train_error_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost (with different local update epochs)")
        plt.legend(local_update_epochs_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_local_update_epochs_training.png')

    # Testing Result
    print(f'!-- Testing Result --!')
    if compareClients is True:
        # Plot the training loss rate between cost history with num_clients
        for i in range(num_clients_list_size):
            plt.plot(test_send_cost_history_total[i], test_loss_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_num_clients_testing.png')

        # Plot the error rate between cost history with local_update_epochs
        for i in range(num_clients_list_size):
            plt.plot(test_send_cost_history_total[i], test_error_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_num_clients_testing.png')

        # Plot the training loss rate between cost history with num_clients per clients
        min_cost_list = test_send_cost_history_total[0]
        for sublist in test_send_cost_history_total:
            if len(sublist) < len(min_cost_list):
                min_cost_list = sublist
        for i in range(num_clients_list_size):
            plt.plot(min_cost_list, test_loss_history_total[i])
        plt.xlabel("Send Cost per clients")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost per clients (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_num_clients_per_clients_testing.png')

        # Plot the training loss rate between cost history with num_clients per clients
        min_cost_list = test_send_cost_history_total[0]
        for sublist in test_send_cost_history_total:
            if len(sublist) < len(min_cost_list):
                min_cost_list = sublist
        for i in range(num_clients_list_size):
            plt.plot(min_cost_list, test_error_history_total[i])
        plt.xlabel("Send Cost per clients")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost per clients (with different number of clients)")
        plt.legend(num_clients_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_num_clients_per_clients_testing.png')
    else:
        # Plot the training loss rate between cost history with local_update_epochs
        for i in range(local_update_epochs_list_size):
            plt.plot(test_send_cost_history_total[i], test_loss_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Training Loss Rate")
        plt.title("Training Loss Rate vs Send Cost (with different local update epochs)")
        plt.legend(local_update_epochs_list)
        plt.show()
        plt.savefig(f'Loss_vs_Send_Cost_{dataset_name}_local_update_epochs_testing.png')

        # Plot the error rate between cost history with local_update_epochs
        for i in range(local_update_epochs_list_size):
            plt.plot(test_send_cost_history_total[i], test_error_history_total[i])
        plt.xlabel("Send Cost")
        plt.ylabel("Error Rate")
        plt.title("Error Rate vs Send Cost (with different local update epochs)")
        plt.legend(local_update_epochs_list)
        plt.show()
        plt.savefig(f'ErrorRate_vs_Send_Cost_{dataset_name}_local_update_epochs_testing.png')

    # Compare training and testing result
    

    # Save the training result

In [ ]:
def simple_global_model_func_CNN():
    if dataset_name == "CIFAR-10 Datset":
        model = ConvolutionalNeuralNetwork_CIFAR10(num_classes_preset)
    else:
        model = ConvolutionalNeuralNetwork(num_classes_preset)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0)
    return model, criterion, optimizer

def simple_client_setup_func_CNN(num_clients, client_model_list, client_optimizer_list, client_criterion_list, client_dataloader_list):
    for client in range(num_clients):
        if dataset_name == "CIFAR-10 Datset":
            client_model_list[client] = ConvolutionalNeuralNetwork_CIFAR10(num_classes_preset)
        else:
            client_model_list[client] = ConvolutionalNeuralNetwork(num_classes_preset)
        client_criterion_list[client] = nn.CrossEntropyLoss()
        client_optimizer_list[client] = torch.optim.SGD(client_model_list[client].parameters(), lr=learning_rate)

In [ ]:
### Experiment for num_clients with CNN model

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
experiment_type = "num_clients"
total_filename = "{}_{}_{}epoch_result".format(dataset_name, experiment_type, num_epochs)

# Define the model parameters
learning_rate = learning_rate_preset
num_epochs = 5
batch_size = batch_size_preset

num_clients_list = [1,2,5]
local_update_epochs_list = [2]

CNN_input_shape = CNN_input_shape_preset

# Experiment
experiment_FedLearn_model(simple_global_model_func_CNN, train_loader, None, num_clients_list, local_update_epochs_list, simple_client_setup_func_CNN, iterate_CNN_model, Federated_Averaging, compareClients=True)

In [ ]:
### Experiment for local_update_epochs with CNN model

# Show Dataset Name
print(f'Current Dataset: {dataset_name}')
experiment_type = "local_update_epochs"
total_filename = "{}_{}_{}epoch_result".format(dataset_name, experiment_type, num_epochs)

# Define the model parameters
learning_rate = learning_rate_preset
num_epochs = 5
batch_size = batch_size_preset

num_clients_list = [2]
local_update_epochs_list = [1,2]

CNN_input_shape = CNN_input_shape_preset

# Experiment
experiment_FedLearn_model(simple_global_model_func_CNN, train_loader, None, num_clients_list, local_update_epochs_list, simple_client_setup_func_CNN, iterate_CNN_model, Federated_Averaging, compareClients=False)

In [ ]:
total_filename = "{}_{}_{}epoch_result".format(dataset_name, experiment_type, num_epochs) #update name of result .npz file
result = np.load(total_filename) #load data into 'result'
print('filename: ', total_filename)
print(result.files) #show attributes inside 'result'

data_train_loss = result['train_loss']    #np.load(total_filename['train_loss'])
data_train_error = result['train_error']    #np.load(total_filename['train_error'])
data_train_send_cost = result['train_send_cost']    #np.load(total_filename['train_send_cost'])
data_train_time = result['train_time']    #np.load(total_filename['train_time'])
data_test_loss = result['test_loss']    #np.load(total_filename['test_loss'])
data_test_error = result['test_error']    #np.load(total_filename['test_error'])
data_test_send_cost = result['test_send_cost']    #np.load(total_filename['test_send_cost'])
data_test_time = result['test_time']    #np.load(total_filename['test_time'])

print("=======TRAIN RESULT=======")
print("train loss: ", data_train_loss)
print("train error: ", data_train_error)
print("train send cost: ", data_train_send_cost)
print("train time: ", data_train_time)
print("=======TEST RESULT=======")
print("test loss: ", data_test_loss)
print("test error: ", data_test_error)
print("test send cost: ", data_test_send_cost)
print("test time: ", data_test_time)